In [ ]:
import os
OPENAI_API_KEY = ""

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.prompts import PromptTemplate
import pickle

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

with open("conf_styles.pickle", "rb") as pr:
    examples = pickle.load(pr)

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

#k값 조정해보기
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), FAISS, k=4
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=
        """
        너는 사용자의 문장을 공자의 말투로 바꿔주는 인공지능이야.
        사용자의 질문을 공자라면 어떻게 말했을지 생각해서 같은 의미의 문장을 답변해줘
        반드시 공자의 말투로 답변해주어야 해.
        """,
    suffix="Input: {input_noun}\n",
    input_variables=["input_noun"],
)

In [ ]:
input_set = {"question":"만약 부를 추구해서 얻을 수 있다면, 천한 일이라도 채찍을 들고 하겠어요. 하지만 부가 추구만으로 얻어지지 않는 것이라면, 제가 좋아하는 일을 하는 편이 낫겠죠.",
             "answer":"부가 만약 추구해서 얻을 수 있는 것이라면, 비록 채찍을 드는 천한 일이라도 나는 하겠다. 그러나 추구해서 얻을 수 없는 일이라면 내가 좋아하는 일을 하겠다."}

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=1.0, model="gpt-3.5-turbo")
chain = LLMChain(llm=chat_model, prompt=similar_prompt, verbose=True)

result = chain(similar_prompt.format(input_noun=input_set["question"]))
print(result["text"])

In [ ]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("embedding_distance", embeddings=OpenAIEmbeddings())
final_string = result["text"].replace("Output: ", "")
score = evaluator.evaluate_strings(prediction=final_string, reference=input_set["answer"])
print(score)